# Initialization

In [ ]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow import keras

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "2021-09 Fall Semester", "ECBM 4040 Neural Network Deep Learning", "Project", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork, Attention56, Attention92
from src.utils import generate_data

# Modelling

In [ ]:
x_train, y_train, x_test, y_test, datagen = generate_data.get_cifar10(rotation_range=20,
                                                                      width_shift_range=0.2,
                                                                      height_shift_range=0.2,
                                                                      horizontal_flip=True,
                                                                      validation_split=0.2)

In [ ]:
INPUT_SHAPE = x_train.shape[1:]
NUM_CLASS = y_train.shape[1]
BATCH_SIZE = 256
N_EPOCH = 10

## Naive Attention Learning vs Residual Attention Learning

### Naive Attention Learning

In [ ]:
ran_model = ResidualAttentionNetwork(learning_type='nal')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 residual_unit (ResidualUnit  multiple                 6432      
 )                                                               
                                                                 
 attention_module (Attention  multiple                 116608    
 Module)                                                         
                                        

In [ ]:
nal_model = ran_model

nal_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = nal_model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                        validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 73s 347ms/step - loss: 2.2965 - accuracy: 0.1332 - val_loss: 2.3498 - val_accuracy: 0.0999
Epoch 2/10
157/157 [==============================] - 50s 320ms/step - loss: 2.1305 - accuracy: 0.2123 - val_loss: 2.4616 - val_accuracy: 0.1332
Epoch 3/10
157/157 [==============================] - 50s 320ms/step - loss: 1.9824 - accuracy: 0.2711 - val_loss: 2.2443 - val_accuracy: 0.2018
Epoch 4/10
157/157 [==============================] - 50s 319ms/step - loss: 1.8942 - accuracy: 0.3067 - val_loss: 1.9455 - val_accuracy: 0.2913
Epoch 5/10
157/157 [==============================] - 50s 320ms/step - loss: 1.8331 - accuracy: 0.3316 - val_loss: 1.9147 - val_accuracy: 0.3196
Epoch 6/10
157/157 [==============================] - 50s 321ms/step - loss: 1.7814 - accuracy: 0.3485 - val_loss: 2.3502 - val_accuracy: 0.3234
Epoch 7/10
157/157 [==============================] - 50s 320ms/step - loss: 1.7305 - accuracy: 0.3649 - val_loss: 1.8189 - val_ac

### Residual Attention Learning

In [ ]:
ran_model = ResidualAttentionNetwork(learning_type='arl')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 residual_unit_33 (ResidualU  multiple                 6432      
 nit)                                                            
                                                                 
 attention_module_3 (Attenti  multiple                 116608    
 onModule)                                                       
                                      

In [ ]:
arl_model = ran_model

arl_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = arl_model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                        validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 70s 343ms/step - loss: 2.1899 - accuracy: 0.1823 - val_loss: 2.6433 - val_accuracy: 0.1032
Epoch 2/10
157/157 [==============================] - 50s 319ms/step - loss: 1.9601 - accuracy: 0.2656 - val_loss: 2.8270 - val_accuracy: 0.1201
Epoch 3/10
157/157 [==============================] - 50s 320ms/step - loss: 1.8131 - accuracy: 0.3185 - val_loss: 2.4663 - val_accuracy: 0.2129
Epoch 4/10
157/157 [==============================] - 51s 321ms/step - loss: 1.7257 - accuracy: 0.3571 - val_loss: 1.7551 - val_accuracy: 0.3414
Epoch 5/10
157/157 [==============================] - 50s 318ms/step - loss: 1.6582 - accuracy: 0.3870 - val_loss: 1.7257 - val_accuracy: 0.3679
Epoch 6/10
157/157 [==============================] - 50s 319ms/step - loss: 1.6070 - accuracy: 0.4070 - val_loss: 1.9173 - val_accuracy: 0.3481
Epoch 7/10
157/157 [==============================] - 50s 320ms/step - loss: 1.5651 - accuracy: 0.4250 - val_loss: 1.6508 - val_ac

## Different number of attention module

In this part, the experiment is done using Residual Attention Learning mechanism

### Attention-56

In [ ]:
ran_model = Attention56()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1728      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple                 75904     
 )                                                               
                                                       

In [ ]:
a56_model = ran_model

a56_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a56_model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                        validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 250s 1s/step - loss: 2.4313 - accuracy: 0.1500 - val_loss: 2.4130 - val_accuracy: 0.0999
Epoch 2/10
157/157 [==============================] - 218s 1s/step - loss: 2.0335 - accuracy: 0.2475 - val_loss: 5.8116 - val_accuracy: 0.1016
Epoch 3/10
157/157 [==============================] - 218s 1s/step - loss: 1.7997 - accuracy: 0.3252 - val_loss: 11.2246 - val_accuracy: 0.1409
Epoch 4/10
157/157 [==============================] - 218s 1s/step - loss: 1.6848 - accuracy: 0.3762 - val_loss: 1.7482 - val_accuracy: 0.3559
Epoch 5/10
157/157 [==============================] - 218s 1s/step - loss: 1.5898 - accuracy: 0.4169 - val_loss: 3.1556 - val_accuracy: 0.2398
Epoch 6/10
157/157 [==============================] - 218s 1s/step - loss: 1.5175 - accuracy: 0.4482 - val_loss: 2.2535 - val_accuracy: 0.3431
Epoch 7/10
 14/157 [=>............................] - ETA: 3:04 - loss: 1.4835 - accuracy: 0.4554

KeyboardInterrupt: ignored

### Attention-92

In [ ]:
ran_model = Attention92()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_139 (Conv2D)         multiple                  1728      
                                                                 
 batch_normalization_107 (Ba  multiple                 256       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple                 75904     
 nit)                                                            
                                                       

In [ ]:
a92_model = ran_model

a92_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a92_model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                        validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                        epochs=N_EPOCH, verbose=1)